## Ading New Features

In [17]:
import pandas as pd
import import_ipynb
from data_cleaning import df_trip 
df_weather= pd.read_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/lagos 2021-01-01 to 2022-12-31.csv')
df_weather=df_weather[['datetime','icon']]
df_weather

,datetime,icon
0,2021-01-01,partly-cloudy-day
1,2021-01-02,rain
2,2021-01-03,rain
3,2021-01-04,rain
4,2021-01-05,rain
...,...,...
725,2022-12-27,partly-cloudy-day
726,2022-12-28,partly-cloudy-day
727,2022-12-29,partly-cloudy-day
728,2022-12-30,partly-cloudy-day


In [ ]:
import pandas as pd
import requests

# Function to get holiday name for a specific date
def get_holiday_name(row):
    year = str(row['st_year'])
    month = str(row['st_month'])
    day = str(row['st_date'])
    api_key = ''
    url = f'https://calendarific.com/api/v2/holidays?&api_key={api_key}&country=NG&year={year}&month={month}&day={day}'
    response = requests.get(url)
    data = response.json()
    
    try:
        # Check if 'holidays' key exists and it's not empty
        if 'holidays' in data['response'] and data['response']['holidays']:
            holiday_name = data['response']['holidays'][0]['name']
        else:
            holiday_name = "No holiday"
    except (KeyError, IndexError):
        # Handle KeyError (if 'response' or 'name' key is missing) or IndexError (if 'holidays' list is empty)
        holiday_name = "No holiday"
    
    return holiday_name

# Extract unique dates from the DataFrame
unique_dates = df_trip[['st_year', 'st_month', 'st_date']].drop_duplicates()

# Get holiday names for these unique dates
unique_dates['holiday_name'] = unique_dates.apply(get_holiday_name, axis=1)

# Create a dictionary for mapping
date_to_holiday = {
    (row['st_year'], row['st_month'], row['st_date']): row['holiday_name']
    for _, row in unique_dates.iterrows()
}

# Map the holiday names back to the original DataFrame
df_trip['holiday_name'] = df_trip.apply(lambda row: date_to_holiday[(row['st_year'], row['st_month'], row['st_date'])], axis=1)

# Display the DataFrame
print(df_trip)


In [20]:
df_trip= pd.read_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/trip.csv')


### Adding Enviromental Data 

In [21]:
df_weather.rename(columns={'datetime': 'End_date'}, inplace=True)
df_trip = pd.merge(df_weather,df_trip, on='End_date')

### Adding Weekday/Weekend data

In [22]:
df_trip['End_date'] = pd.to_datetime(df_trip['End_date'])

# Function to determine if a date is a weekday or weekend
def determine_weekday_or_weekend(date):
    if date.weekday() < 5:  # Weekday is 0-4, Monday to Friday
        return 'Weekday'
    else:  # Weekend is 5-6, Saturday and Sunday
        return 'Weekend'

# Apply the function to create a new column
df_trip['day_type'] = df_trip['End_date'].apply(determine_weekday_or_weekend)

df_trip

,End_date,icon,Unnamed: 0,Trip ID,lat_org,lon_org,lat_des,lon_des,Start_time,End_time,st_year,st_month,st_date,holiday_name,day_type
0,2021-07-01,rain,0,391996,6.508813,3.377403,6.650970,3.345031,07:28:04,07:29:37,2021,7,1,No holiday,Weekday
1,2021-07-01,rain,1,391997,6.431671,3.455538,6.428081,3.472189,06:38:04,07:07:28,2021,7,1,No holiday,Weekday
2,2021-07-01,rain,2,391998,6.631679,3.338898,6.508324,3.359040,06:21:02,07:02:23,2021,7,1,No holiday,Weekday
3,2021-07-01,rain,3,391999,6.572757,3.367708,6.584881,3.361407,07:16:07,07:29:42,2021,7,1,No holiday,Weekday
4,2021-07-01,rain,4,392001,6.601042,3.276634,6.450107,3.391615,09:30:59,09:34:36,2021,7,1,No holiday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534363,2022-05-19,rain,431404,1345209,6.635022,3.338230,6.460184,3.193621,10:37:32,10:51:50,2022,5,19,No holiday,Weekday
534364,2022-05-30,partly-cloudy-day,524513,1604908,6.509019,3.387048,6.626393,3.325983,13:26:33,13:39:24,2022,5,30,No holiday,Weekday
534365,2022-06-03,rain,490664,1488991,6.444520,3.547642,6.606708,3.318793,16:20:25,16:23:19,2022,6,3,No holiday,Weekday
534366,2022-06-04,partly-cloudy-day,462926,1420310,6.452809,3.433489,6.557231,3.290836,15:17:38,08:49:35,2022,6,3,No holiday,Weekend


### Calculating speed

In [23]:
from math import radians, cos, sin, sqrt, atan2

# Haversine function to calculate the distance between two points
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance

In [24]:
df_trip['trip_start_time'] = df_trip[['st_year', 'st_month', 'st_date']].astype(str).agg('-'.join, axis=1)
df_trip = df_trip.drop(columns=['st_year', 'st_month', 'st_date'])
df_trip['trip_end_time'] = df_trip[['End_date', 'End_time']].astype(str).agg(' '.join, axis=1)
df_trip = df_trip.drop(columns=['End_date', 'End_time'])
df_trip['trip_start_time'] = df_trip[['trip_start_time', 'Start_time']].astype(str).agg(' '.join, axis=1)
df_trip = df_trip.drop(columns=['Start_time'])

df_trip

,icon,Unnamed: 0,Trip ID,lat_org,lon_org,lat_des,lon_des,holiday_name,day_type,trip_start_time,trip_end_time
0,rain,0,391996,6.508813,3.377403,6.650970,3.345031,No holiday,Weekday,2021-7-1 07:28:04,2021-07-01 07:29:37
1,rain,1,391997,6.431671,3.455538,6.428081,3.472189,No holiday,Weekday,2021-7-1 06:38:04,2021-07-01 07:07:28
2,rain,2,391998,6.631679,3.338898,6.508324,3.359040,No holiday,Weekday,2021-7-1 06:21:02,2021-07-01 07:02:23
3,rain,3,391999,6.572757,3.367708,6.584881,3.361407,No holiday,Weekday,2021-7-1 07:16:07,2021-07-01 07:29:42
4,rain,4,392001,6.601042,3.276634,6.450107,3.391615,No holiday,Weekday,2021-7-1 09:30:59,2021-07-01 09:34:36
...,...,...,...,...,...,...,...,...,...,...,...
534363,rain,431404,1345209,6.635022,3.338230,6.460184,3.193621,No holiday,Weekday,2022-5-19 10:37:32,2022-05-19 10:51:50
534364,partly-cloudy-day,524513,1604908,6.509019,3.387048,6.626393,3.325983,No holiday,Weekday,2022-5-30 13:26:33,2022-05-30 13:39:24
534365,rain,490664,1488991,6.444520,3.547642,6.606708,3.318793,No holiday,Weekday,2022-6-3 16:20:25,2022-06-03 16:23:19
534366,partly-cloudy-day,462926,1420310,6.452809,3.433489,6.557231,3.290836,No holiday,Weekend,2022-6-3 15:17:38,2022-06-04 08:49:35


In [25]:
df_trip ['trip_end_time'] = pd.to_datetime(df_trip['trip_end_time'])
df_trip ['trip_start_time'] = pd.to_datetime(df_trip['trip_start_time'])

In [26]:
# Function to calculate speed
def calculate_speed(row):
    distance = haversine(row['lat_org'], row['lon_org'], row['lat_des'], row['lon_des'])
    time_diff = (row['trip_end_time'] - row['trip_start_time']).total_seconds() / 3600  # Convert time difference to hours
    # Debugging output
    # print(f"Start: ({row['lat_org']}, {row['lon_org']}), End: ({row['lat_des']}, {row['lon_des']})")
    # print(f"Distance: {distance:.2f} km, Time difference: {time_diff:.2f} hours")
    
    if time_diff == 0:
        return 0  # To handle division by zero
    speed = distance / time_diff  # Speed in km/h
    return speed

# Apply the function to calculate speed
df_trip['speed_kmh'] = df_trip.apply(calculate_speed, axis=1)


In [28]:
df_trip

,icon,Unnamed: 0,Trip ID,lat_org,lon_org,lat_des,lon_des,holiday_name,day_type,trip_start_time,trip_end_time,speed_kmh
0,rain,0,391996,6.508813,3.377403,6.650970,3.345031,No holiday,Weekday,2021-07-01 07:28:04,2021-07-01 07:29:37,627.350298
1,rain,1,391997,6.431671,3.455538,6.428081,3.472189,No holiday,Weekday,2021-07-01 06:38:04,2021-07-01 07:07:28,3.842195
2,rain,2,391998,6.631679,3.338898,6.508324,3.359040,No holiday,Weekday,2021-07-01 06:21:02,2021-07-01 07:02:23,20.163153
3,rain,3,391999,6.572757,3.367708,6.584881,3.361407,No holiday,Weekday,2021-07-01 07:16:07,2021-07-01 07:29:42,6.701681
4,rain,4,392001,6.601042,3.276634,6.450107,3.391615,No holiday,Weekday,2021-07-01 09:30:59,2021-07-01 09:34:36,349.187758
...,...,...,...,...,...,...,...,...,...,...,...,...
534363,rain,431404,1345209,6.635022,3.338230,6.460184,3.193621,No holiday,Weekday,2022-05-19 10:37:32,2022-05-19 10:51:50,105.577035
534364,partly-cloudy-day,524513,1604908,6.509019,3.387048,6.626393,3.325983,No holiday,Weekday,2022-05-30 13:26:33,2022-05-30 13:39:24,68.598589
534365,rain,490664,1488991,6.444520,3.547642,6.606708,3.318793,No holiday,Weekday,2022-06-03 16:20:25,2022-06-03 16:23:19,642.519826
534366,partly-cloudy-day,462926,1420310,6.452809,3.433489,6.557231,3.290836,No holiday,Weekend,2022-06-03 15:17:38,2022-06-04 08:49:35,1.116533


In [27]:
lower_limit = 24  # Lower limit in km/h
upper_limit = 113  # Upper limit in km/h

# Create a filter for abnormal speeds
abnormal_speeds_filter = (df_trip['speed_kmh'] < lower_limit) | (df_trip['speed_kmh'] > upper_limit)

# Apply the filter to the DataFrame
filtered_df = df_trip[~abnormal_speeds_filter]  # Invert the filter to keep normal speeds

filtered_df

,icon,Unnamed: 0,Trip ID,lat_org,lon_org,lat_des,lon_des,holiday_name,day_type,trip_start_time,trip_end_time,speed_kmh
8,rain,8,392014,6.499156,3.358517,6.428091,3.515717,No holiday,Weekday,2021-07-01 06:43:07,2021-07-01 07:27:24,25.854164
10,rain,10,392018,6.446071,3.478035,6.641426,3.363795,No holiday,Weekday,2021-07-01 06:51:28,2021-07-01 07:45:09,28.078387
17,rain,17,392033,6.521411,3.367289,6.638375,3.333120,No holiday,Weekday,2021-07-01 07:27:43,2021-07-01 07:58:24,26.481685
56,rain,57,392117,6.468855,3.634946,6.450564,3.518006,No holiday,Weekday,2021-07-01 08:03:59,2021-07-01 08:29:00,31.370517
63,rain,64,392136,6.460126,3.192317,6.603128,3.379186,No holiday,Weekday,2021-07-01 10:46:20,2021-07-01 11:39:57,29.160371
...,...,...,...,...,...,...,...,...,...,...,...,...
534298,partly-cloudy-day,520242,1580612,6.627720,3.487806,6.458784,3.415552,No holiday,Weekend,2022-01-29 16:01:34,2022-01-29 16:40:22,31.562404
534350,partly-cloudy-day,283245,1026231,6.432499,3.553026,6.436349,3.467128,No holiday,Weekday,2022-04-11 11:20:21,2022-04-11 11:41:06,27.472463
534354,rain,517946,1570709,6.530983,3.356092,6.647137,3.514775,Holy Saturday,Weekend,2022-04-16 13:30:42,2022-04-16 14:12:23,31.340200
534363,rain,431404,1345209,6.635022,3.338230,6.460184,3.193621,No holiday,Weekday,2022-05-19 10:37:32,2022-05-19 10:51:50,105.577035
